In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

import networkx as nx
import scglue
from itertools import chain
import itertools
from tqdm import tqdm

import scanpy as sc
import anndata as ad
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing as pp

import time
from tqdm import tqdm
import seaborn as sns
import scib_metrics
import helper_functions as fcts

/dss/dsshome1/01/di93bur/miniconda3/envs/scglue_env_2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
i = 's1d2'
atac = sc.read('data/'+i+'/atac-glue.h5ad')
rna = sc.read('data/'+i+'/rna-glue.h5ad')
guidance = nx.read_graphml('data/'+i+'/guidance.graphml.gz')

In [4]:
scglue.models.configure_dataset(
    rna, "NB", use_highly_variable=True,
    use_layer="counts", use_rep="X_pca",
    use_obs_names=True
)

In [5]:
scglue.models.configure_dataset(
    atac, "NB", use_highly_variable=True,
    use_layer="counts", use_rep="X_lsi_red",
    use_obs_names=True
)

In [6]:
guidance_hvf = guidance.subgraph(chain(
    rna.var.query("highly_variable").index,
    atac.var.query("highly_variable").index
)).copy()

In [7]:
glue = scglue.models.fit_SCGLUE(
    {"rna": rna, "atac": atac}, guidance_hvf,
    model=scglue.models.PairedSCGLUEModel,
    fit_kws={"directory": "glue"}
)

[INFO] fit_SCGLUE: Pretraining SCGLUE model...


[WARNING] PairedSCGLUEModel: It is recommended that `use_rep` dimensionality be equal or larger than `latent_dim`.


[INFO] autodevice: Using CPU as computation device.
[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] PairedSCGLUEModel: Setting `graph_batch_size` = 3437
[INFO] PairedSCGLUEModel: Setting `max_epochs` = 507
[INFO] PairedSCGLUEModel: Setting `patience` = 43
[INFO] PairedSCGLUEModel: Setting `reduce_lr_patience` = 22
[INFO] PairedSCGLUETrainer: Using training directory: "glue/pretrain"
[INFO] PairedSCGLUETrainer: [Epoch 10] train={'g_nll': 0.624, 'g_kl': 0.034, 'g_elbo': 0.659, 'x_rna_nll': 0.528, 'x_rna_kl': 0.014, 'x_rna_elbo': 0.542, 'x_atac_nll': 0.671, 'x_atac_kl': 0.007, 'x_atac_elbo': 0.678, 'dsc_loss': 0.683, 'vae_loss': 1.301, 'gen_loss': 1.267, 'joint_cross_loss': 1.197, 'real_cross_loss': 1.216, 'cos_loss': 0.342}, val={'g_nll': 0.626, 'g_kl': 0.035, 'g_elbo': 0.661, 'x_rna_nll': 0.521, 'x_rna_kl': 0.012, 'x_rna_elbo': 0.533, 'x_atac

2023-05-05 14:04:56,054 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


[INFO] EarlyStopping: Restoring checkpoint "112"...
[INFO] EarlyStopping: Restoring checkpoint "112"...
[INFO] fit_SCGLUE: Estimating balancing weight...
[INFO] estimate_balancing_weight: Clustering cells...
[INFO] estimate_balancing_weight: Matching clusters...
[INFO] estimate_balancing_weight: Matching array shape = (16, 18)...
[INFO] estimate_balancing_weight: Estimating balancing weight...


...

[INFO] fit_SCGLUE: Fine-tuning SCGLUE model...


[WARNING] PairedSCGLUEModel: It is recommended that `use_rep` dimensionality be equal or larger than `latent_dim`.


[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] PairedSCGLUEModel: Setting `graph_batch_size` = 3437
[INFO] PairedSCGLUEModel: Setting `align_burnin` = 85
[INFO] PairedSCGLUEModel: Setting `max_epochs` = 507
[INFO] PairedSCGLUEModel: Setting `patience` = 43
[INFO] PairedSCGLUEModel: Setting `reduce_lr_patience` = 22
[INFO] PairedSCGLUETrainer: Using training directory: "glue/fine-tune"
[INFO] PairedSCGLUETrainer: [Epoch 10] train={'g_nll': 0.442, 'g_kl': 0.036, 'g_elbo': 0.479, 'x_rna_nll': 0.504, 'x_rna_kl': 0.009, 'x_rna_elbo': 0.513, 'x_atac_nll': 0.653, 'x_atac_kl': 0.003, 'x_atac_elbo': 0.657, 'dsc_loss': 0.691, 'vae_loss': 1.246, 'gen_loss': 1.211, 'joint_cross_loss': 1.163, 'real_cross_loss': 1.189, 'cos_loss': 0.461}, val={'g_nll': 0.44, 'g_kl': 0.036, 'g_elbo': 0.476, 'x_rna_nll': 0.49, 'x_rna_kl': 0.009, 'x_rna_elbo': 0.499, 'x_ata

2023-05-05 14:35:06,669 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


[INFO] EarlyStopping: Restoring checkpoint "161"...
[INFO] EarlyStopping: Restoring checkpoint "161"...


In [8]:
rna.obsm["X_glue"] = glue.encode_data("rna", rna)
atac.obsm["X_glue"] = glue.encode_data("atac", atac)

In [11]:
adata = sc.concat([rna, atac], join = 'outer', label='batch', index_unique = '-', keys=["RNA", "ATAC"])

In [12]:
adata

AnnData object with n_obs × n_vars = 13480 × 22805
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'n_counts', '_scvi_batch', '_scvi_labels', 'cluster', 'balancing_weight'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap', 'X_mde', 'X_pca', 'X_scVI', 'X_umap', 'X_glue', 'ATAC_lsi_red_new', 'X_PeakVI', 'X_lsi', 'X_lsi_red', 'X_poissonVI'
    layers: 'counts'

In [10]:
f = fcts.foscttm(rna.obsm["X_glue"], atac.obsm["X_glue"])
print("The average FOSCTTM for the alignment is: ", f)

The average FOSCTTM for the alignment is:  0.0422


In [14]:
import scib

In [16]:
# Evaluation metrics
f = fcts.foscttm(rna.obsm["X_glue"], atac.obsm["X_glue"])
lta = fcts.transfer_accuracy(rna.obsm["X_glue"], atac.obsm["X_glue"], rna.obs['cell_type'], atac.obs['cell_type'])
alg_score = fcts.alignment_score(rna.obsm["X_glue"], atac.obsm["X_glue"])
rmse = fcts.calculate_rmse(rna.obsm["X_glue"], atac.obsm["X_glue"])

# Bio-conservation metrics
sc.pp.neighbors(adata, use_rep='X_glue')
scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)
nmi = scib.me.nmi(adata, cluster_key='cluster_key', label_key='cell_type')
asw_label = scib.me.silhouette(adata, label_key='cell_type', embed='X_glue')
ari = scib.me.ari(adata, cluster_key='cluster_key', label_key='cell_type')

# Batch correction metrics
asw_batch = scib.me.silhouette_batch(adata,batch_key='batch',label_key='cell_type',embed='X_glue',verbose=False)
graph_connectivity = scib.me.graph_connectivity(adata, label_key='cell_type')

/tmp/ipykernel_413349/3849138315.py:9: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)
/dss/dsshome1/01/di93bur/miniconda3/envs/scglue_env_2/lib/python3.9/site-packages/scanpy/tools/_louvain.py:138: DeprecationWarning: This package has been superseded by the `leidenalg` package and will no longer be maintained. Please upgrade to the `leidenalg` package.
  import louvain


In [19]:
results = []
results.append([i, f, lta, alg_score, rmse, nmi, asw_label, ari, asw_batch, graph_connectivity])

In [21]:
df_results = pd.DataFrame(
    results,
    columns = ["donor", "foscttm", "label_transfer_accuracy", "alignment_score", "rmse", "nmi", "asw_label", "ari", "asw_batch", "graph_connectivity"]
)

In [22]:
df_results

,donor,foscttm,label_transfer_accuracy,alignment_score,rmse,nmi,asw_label,ari,asw_batch,graph_connectivity
0,s1d2,0.0422,0.842285,0.956476,0.169282,0.700974,0.585814,0.648648,0.910727,0.951778


In [23]:
df_results.to_csv('output/scglue_2.csv', index = False)

In [24]:
for i in tqdm(['s1d3', 's2d1', 's2d4', 's2d5', 's3d10', 's3d3', 's3d6', 's3d7', 's4d1', 's4d8', 's4d9']):
    atac = sc.read('data/'+i+'/atac-glue.h5ad')
    rna = sc.read('data/'+i+'/rna-glue.h5ad')
    guidance = nx.read_graphml('data/'+i+'/guidance.graphml.gz')
    
    scglue.models.configure_dataset(rna, "NB", use_highly_variable=True,use_layer="counts", use_rep="X_pca",use_obs_names=True)
    scglue.models.configure_dataset(atac, "NB", use_highly_variable=True,use_layer="counts", use_rep="X_lsi_red",use_obs_names=True)
    guidance_hvf = guidance.subgraph(chain(rna.var.query("highly_variable").index,atac.var.query("highly_variable").index)).copy()
    
    glue = scglue.models.fit_SCGLUE({"rna": rna, "atac": atac}, guidance_hvf,model=scglue.models.PairedSCGLUEModel,fit_kws={"directory": "glue"})
    
    rna.obsm["X_glue"] = glue.encode_data("rna", rna)
    atac.obsm["X_glue"] = glue.encode_data("atac", atac)
    adata = sc.concat([rna, atac], join = 'outer', label='batch', index_unique = '-', keys=["RNA", "ATAC"])
    
    # Evaluation metrics
    f = fcts.foscttm(rna.obsm["X_glue"], atac.obsm["X_glue"])
    lta = fcts.transfer_accuracy(rna.obsm["X_glue"], atac.obsm["X_glue"], rna.obs['cell_type'], atac.obs['cell_type'])
    alg_score = fcts.alignment_score(rna.obsm["X_glue"], atac.obsm["X_glue"])
    rmse = fcts.calculate_rmse(rna.obsm["X_glue"], atac.obsm["X_glue"])

    # Bio-conservation metrics
    sc.pp.neighbors(adata, use_rep='X_glue')
    scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)
    nmi = scib.me.nmi(adata, cluster_key='cluster_key', label_key='cell_type')
    asw_label = scib.me.silhouette(adata, label_key='cell_type', embed='X_glue')
    ari = scib.me.ari(adata, cluster_key='cluster_key', label_key='cell_type')

    # Batch correction metrics
    asw_batch = scib.me.silhouette_batch(adata,batch_key='batch',label_key='cell_type',embed='X_glue',verbose=False)
    graph_connectivity = scib.me.graph_connectivity(adata, label_key='cell_type')
    
    results.append([i, f, lta, alg_score, rmse, nmi, asw_label, ari, asw_batch, graph_connectivity])

  0%|          | 0/11 [00:00<?, ?it/s]

[INFO] fit_SCGLUE: Pretraining SCGLUE model...


[WARNING] PairedSCGLUEModel: It is recommended that `use_rep` dimensionality be equal or larger than `latent_dim`.


[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] PairedSCGLUEModel: Setting `graph_batch_size` = 3438
[INFO] PairedSCGLUEModel: Setting `max_epochs` = 798
[INFO] PairedSCGLUEModel: Setting `patience` = 67
[INFO] PairedSCGLUEModel: Setting `reduce_lr_patience` = 34
[INFO] PairedSCGLUETrainer: Using training directory: "glue/pretrain"
[INFO] PairedSCGLUETrainer: [Epoch 10] train={'g_nll': 0.586, 'g_kl': 0.025, 'g_elbo': 0.611, 'x_rna_nll': 0.509, 'x_rna_kl': 0.021, 'x_rna_elbo': 0.529, 'x_atac_nll': 0.699, 'x_atac_kl': 0.013, 'x_atac_elbo': 0.712, 'dsc_loss': 0.66, 'vae_loss': 1.32, 'gen_loss': 1.287, 'joint_cross_loss': 1.206, 'real_cross_loss': 1.232, 'cos_loss': 0.279}, val={'g_nll': 0.589, 'g_kl': 0.026, 'g_elbo': 0.615, 'x_rna_nll': 0.51, 'x_rna_kl': 0.02, 'x_rna_elbo': 0.53, 'x_atac_nll': 0.705, 'x_atac_kl': 0.012, 'x_atac_elbo': 0.717, '

2023-05-05 15:55:54,432 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


...

[WARNING] PairedSCGLUEModel: It is recommended that `use_rep` dimensionality be equal or larger than `latent_dim`.


[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] PairedSCGLUEModel: Setting `graph_batch_size` = 4560
[INFO] PairedSCGLUEModel: Setting `max_epochs` = 790
[INFO] PairedSCGLUEModel: Setting `patience` = 66
[INFO] PairedSCGLUEModel: Setting `reduce_lr_patience` = 33
[INFO] PairedSCGLUETrainer: Using training directory: "glue/pretrain"
[INFO] PairedSCGLUETrainer: [Epoch 10] train={'g_nll': 0.613, 'g_kl': 0.024, 'g_elbo': 0.637, 'x_rna_nll': 0.446, 'x_rna_kl': 0.014, 'x_rna_elbo': 0.459, 'x_atac_nll': 0.638, 'x_atac_kl': 0.009, 'x_atac_elbo': 0.647, 'dsc_loss': 0.681, 'vae_loss': 1.182, 'gen_loss': 1.148, 'joint_cross_loss': 1.081, 'real_cross_loss': 1.095, 'cos_loss': 0.303}, val={'g_nll': 0.613, 'g_kl': 0.024, 'g_elbo': 0.637, 'x_rna_nll': 0.451, 'x_rna_kl': 0.012, 'x_rna_elbo': 0.463, 'x_atac_nll': 0.671, 'x_atac_kl': 0.009, 'x_atac_elbo': 0.6

Engine run is terminating due to exception: 


[INFO] PairedSCGLUETrainer: Stopping training due to user interrupt...


/dss/dsshome1/01/di93bur/miniconda3/envs/scglue_env_2/lib/python3.9/multiprocessing/popen_fork.py:66: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f3dd8395b20>
  self.pid = os.fork()
 45%|████▌     | 5/11 [8:38:51<10:22:37, 6226.27s/it]

KeyboardInterrupt



In [26]:
df_results = pd.DataFrame(
    results,
    columns = ["donor", "foscttm", "label_transfer_accuracy", "alignment_score", "rmse", "nmi", "asw_label", "ari", "asw_batch", "graph_connectivity"]
)

In [28]:
df_results.to_csv('output/scglue_2.csv', index = False)

In [29]:
for i in tqdm(['s3d3', 's3d6']):
    atac = sc.read('data/'+i+'/atac-glue.h5ad')
    rna = sc.read('data/'+i+'/rna-glue.h5ad')
    guidance = nx.read_graphml('data/'+i+'/guidance.graphml.gz')
    
    scglue.models.configure_dataset(rna, "NB", use_highly_variable=True,use_layer="counts", use_rep="X_pca",use_obs_names=True)
    scglue.models.configure_dataset(atac, "NB", use_highly_variable=True,use_layer="counts", use_rep="X_lsi_red",use_obs_names=True)
    guidance_hvf = guidance.subgraph(chain(rna.var.query("highly_variable").index,atac.var.query("highly_variable").index)).copy()
    
    glue = scglue.models.fit_SCGLUE({"rna": rna, "atac": atac}, guidance_hvf,model=scglue.models.PairedSCGLUEModel,fit_kws={"directory": "glue"})
    
    rna.obsm["X_glue"] = glue.encode_data("rna", rna)
    atac.obsm["X_glue"] = glue.encode_data("atac", atac)
    adata = sc.concat([rna, atac], join = 'outer', label='batch', index_unique = '-', keys=["RNA", "ATAC"])
    
    # Evaluation metrics
    f = fcts.foscttm(rna.obsm["X_glue"], atac.obsm["X_glue"])
    lta = fcts.transfer_accuracy(rna.obsm["X_glue"], atac.obsm["X_glue"], rna.obs['cell_type'], atac.obs['cell_type'])
    alg_score = fcts.alignment_score(rna.obsm["X_glue"], atac.obsm["X_glue"])
    rmse = fcts.calculate_rmse(rna.obsm["X_glue"], atac.obsm["X_glue"])

    # Bio-conservation metrics
    sc.pp.neighbors(adata, use_rep='X_glue')
    scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)
    nmi = scib.me.nmi(adata, cluster_key='cluster_key', label_key='cell_type')
    asw_label = scib.me.silhouette(adata, label_key='cell_type', embed='X_glue')
    ari = scib.me.ari(adata, cluster_key='cluster_key', label_key='cell_type')

    # Batch correction metrics
    asw_batch = scib.me.silhouette_batch(adata,batch_key='batch',label_key='cell_type',embed='X_glue',verbose=False)
    graph_connectivity = scib.me.graph_connectivity(adata, label_key='cell_type')
    
    results.append([i, f, lta, alg_score, rmse, nmi, asw_label, ari, asw_batch, graph_connectivity])
    df_results = pd.DataFrame(
        results,
        columns = ["donor", "foscttm", "label_transfer_accuracy", "alignment_score", "rmse", "nmi", "asw_label", "ari", "asw_batch", "graph_connectivity"]
    )
    df_results.to_csv('output/scglue_2.csv', index = False)

  0%|          | 0/2 [00:00<?, ?it/s]

[INFO] fit_SCGLUE: Pretraining SCGLUE model...


[WARNING] PairedSCGLUEModel: It is recommended that `use_rep` dimensionality be equal or larger than `latent_dim`.


[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] PairedSCGLUEModel: Setting `graph_batch_size` = 4560
[INFO] PairedSCGLUEModel: Setting `max_epochs` = 790
[INFO] PairedSCGLUEModel: Setting `patience` = 66
[INFO] PairedSCGLUEModel: Setting `reduce_lr_patience` = 33
[INFO] PairedSCGLUETrainer: Using training directory: "glue/pretrain"
[INFO] PairedSCGLUETrainer: [Epoch 10] train={'g_nll': 0.613, 'g_kl': 0.024, 'g_elbo': 0.637, 'x_rna_nll': 0.446, 'x_rna_kl': 0.014, 'x_rna_elbo': 0.459, 'x_atac_nll': 0.638, 'x_atac_kl': 0.009, 'x_atac_elbo': 0.647, 'dsc_loss': 0.681, 'vae_loss': 1.182, 'gen_loss': 1.148, 'joint_cross_loss': 1.081, 'real_cross_loss': 1.095, 'cos_loss': 0.304}, val={'g_nll': 0.613, 'g_kl': 0.024, 'g_elbo': 0.637, 'x_rna_nll': 0.451, 'x_rna_kl': 0.012, 'x_rna_elbo': 0.463, 'x_atac_nll': 0.671, 'x_atac_kl': 0.009, 'x_atac_elbo': 0.6

2023-05-06 00:12:08,611 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


...

/dss/dsshome1/01/di93bur/miniconda3/envs/scglue_env_2/lib/python3.9/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/dss/dsshome1/01/di93bur/miniconda3/envs/scglue_env_2/lib/python3.9/site-packages/anndata/_core/anndata.py:117: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
[WARNING] PairedSCGLUEModel: It is recommended that `use_rep` dimensionality be equal or larger than `latent_dim`.


[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] PairedSCGLUEModel: Setting `graph_batch_size` = 3398
[INFO] PairedSCGLUEModel: Setting `align_burnin` = 339
[INFO] PairedSCGLUEModel: Setting `max_epochs` = 2033
[INFO] PairedSCGLUEModel: Setting `patience` = 170
[INFO] PairedSCGLUEModel: Setting `reduce_lr_patience` = 85
[INFO] PairedSCGLUETrainer: Using training directory: "glue/fine-tune"
[INFO] PairedSCGLUETrainer: [Epoch 10] train={'g_nll': 0.446, 'g_kl': 0.04, 'g_elbo': 0.486, 'x_rna_nll': 0.477, 'x_rna_kl': 0.01, 'x_rna_elbo': 0.487, 'x_atac_nll': 0.551, 'x_atac_kl': 0.004, 'x_atac_elbo': 0.555, 'dsc_loss': 0.697, 'vae_loss': 1.112, 'gen_loss': 1.078, 'joint_cross_loss': 1.035, 'real_cross_loss': 1.068, 'cos_loss': 0.48}, val={'g_nll': 0.442, 'g_kl': 0.04, 'g_elbo': 0.482, 'x_rna_nll': 0.489, 'x_rna_kl': 0.01, 'x_rna_elbo': 0.499, 'x_ata

2023-05-06 04:33:54,239 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


[INFO] EarlyStopping: Restoring checkpoint "664"...
[INFO] EarlyStopping: Restoring checkpoint "664"...


/dss/dsshome1/01/di93bur/miniconda3/envs/scglue_env_2/lib/python3.9/multiprocessing/popen_fork.py:66: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f3dd8395b20>
  self.pid = os.fork()
/tmp/ipykernel_413349/2621457291.py:24: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)
100%|██████████| 2/2 [4:49:05<00:00, 8672.54s/it]  


In [30]:
df_results = pd.DataFrame(
    results,
    columns = ["donor", "foscttm", "label_transfer_accuracy", "alignment_score", "rmse", "nmi", "asw_label", "ari", "asw_batch", "graph_connectivity"]
)
df_results.to_csv('output/scglue_2.csv', index = False)

In [31]:
df_results

,donor,foscttm,label_transfer_accuracy,alignment_score,rmse,nmi,asw_label,ari,asw_batch,graph_connectivity
0,s1d2,0.0422,0.842285,0.956476,0.169282,0.700974,0.585814,0.648648,0.910727,0.951778
1,s1d3,0.0326,0.843421,0.968863,0.149477,0.718436,0.570090,0.513514,0.937366,0.972907
2,s2d1,0.0815,0.860190,0.900669,0.168181,0.724143,0.582817,0.735471,0.888634,0.905477
3,s2d4,0.0582,0.872689,0.931155,0.125049,0.751659,0.614832,0.707645,0.900886,0.927510
4,s2d5,0.1006,0.902758,0.874909,0.138881,0.773474,0.620914,0.814020,0.868497,0.923431
5,s3d10,0.0426,0.869046,0.931326,0.134223,0.795790,0.625322,0.720174,0.935331,0.967390
6,s3d3,0.0356,0.851561,0.971671,0.172021,0.779735,0.590462,0.642080,0.958223,0.972580
7,s3d6,0.0859,0.814175,0.970419,0.260126,0.640087,0.538489,0.572510,0.920650,0.944417


In [ ]:
for i in tqdm(['s4d8', 's4d9']):
    atac = sc.read('data/'+i+'/atac-glue.h5ad')
    rna = sc.read('data/'+i+'/rna-glue.h5ad')
    guidance = nx.read_graphml('data/'+i+'/guidance.graphml.gz')
    
    scglue.models.configure_dataset(rna, "NB", use_highly_variable=True,use_layer="counts", use_rep="X_pca",use_obs_names=True)
    scglue.models.configure_dataset(atac, "NB", use_highly_variable=True,use_layer="counts", use_rep="X_lsi_red",use_obs_names=True)
    guidance_hvf = guidance.subgraph(chain(rna.var.query("highly_variable").index,atac.var.query("highly_variable").index)).copy()
    
    glue = scglue.models.fit_SCGLUE({"rna": rna, "atac": atac}, guidance_hvf,model=scglue.models.PairedSCGLUEModel,fit_kws={"directory": "glue"})
    
    rna.obsm["X_glue"] = glue.encode_data("rna", rna)
    atac.obsm["X_glue"] = glue.encode_data("atac", atac)
    adata = sc.concat([rna, atac], join = 'outer', label='batch', index_unique = '-', keys=["RNA", "ATAC"])
    
    # Evaluation metrics
    f = fcts.foscttm(rna.obsm["X_glue"], atac.obsm["X_glue"])
    lta = fcts.transfer_accuracy(rna.obsm["X_glue"], atac.obsm["X_glue"], rna.obs['cell_type'], atac.obs['cell_type'])
    alg_score = fcts.alignment_score(rna.obsm["X_glue"], atac.obsm["X_glue"])
    rmse = fcts.calculate_rmse(rna.obsm["X_glue"], atac.obsm["X_glue"])

    # Bio-conservation metrics
    sc.pp.neighbors(adata, use_rep='X_glue')
    scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)
    nmi = scib.me.nmi(adata, cluster_key='cluster_key', label_key='cell_type')
    asw_label = scib.me.silhouette(adata, label_key='cell_type', embed='X_glue')
    ari = scib.me.ari(adata, cluster_key='cluster_key', label_key='cell_type')

    # Batch correction metrics
    asw_batch = scib.me.silhouette_batch(adata,batch_key='batch',label_key='cell_type',embed='X_glue',verbose=False)
    graph_connectivity = scib.me.graph_connectivity(adata, label_key='cell_type')
    
    results.append([i, f, lta, alg_score, rmse, nmi, asw_label, ari, asw_batch, graph_connectivity])
    df_results = pd.DataFrame(
        results,
        columns = ["donor", "foscttm", "label_transfer_accuracy", "alignment_score", "rmse", "nmi", "asw_label", "ari", "asw_batch", "graph_connectivity"]
    )
    df_results.to_csv('output/scglue_2.csv', index = False)

  0%|          | 0/2 [00:00<?, ?it/s]

[INFO] fit_SCGLUE: Pretraining SCGLUE model...


[WARNING] PairedSCGLUEModel: It is recommended that `use_rep` dimensionality be equal or larger than `latent_dim`.


[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] PairedSCGLUEModel: Setting `graph_batch_size` = 4337
[INFO] PairedSCGLUEModel: Setting `max_epochs` = 346
[INFO] PairedSCGLUEModel: Setting `patience` = 29
[INFO] PairedSCGLUEModel: Setting `reduce_lr_patience` = 15
[INFO] PairedSCGLUETrainer: Using training directory: "glue/pretrain"
[INFO] PairedSCGLUETrainer: [Epoch 10] train={'g_nll': 0.604, 'g_kl': 0.036, 'g_elbo': 0.64, 'x_rna_nll': 0.401, 'x_rna_kl': 0.008, 'x_rna_elbo': 0.408, 'x_atac_nll': 0.605, 'x_atac_kl': 0.004, 'x_atac_elbo': 0.609, 'dsc_loss': 0.691, 'vae_loss': 1.091, 'gen_loss': 1.057, 'joint_cross_loss': 1.004, 'real_cross_loss': 1.014, 'cos_loss': 0.408}, val={'g_nll': 0.598, 'g_kl': 0.036, 'g_elbo': 0.634, 'x_rna_nll': 0.405, 'x_rna_kl': 0.007, 'x_rna_elbo': 0.412, 'x_atac_nll': 0.605, 'x_atac_kl': 0.004, 'x_atac_elbo': 0.60

2023-05-06 12:50:08,987 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training


...

[INFO] EarlyStopping: Restoring checkpoint "94"...
[INFO] EarlyStopping: Restoring checkpoint "94"...


/dss/dsshome1/01/di93bur/miniconda3/envs/scglue_env_2/lib/python3.9/multiprocessing/popen_fork.py:66: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f3dd8395b20>
  self.pid = os.fork()
/tmp/ipykernel_413349/2845210802.py:24: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)
 50%|█████     | 1/2 [1:21:51<1:21:51, 4911.48s/it]

[INFO] fit_SCGLUE: Pretraining SCGLUE model...


[WARNING] PairedSCGLUEModel: It is recommended that `use_rep` dimensionality be equal or larger than `latent_dim`.


[INFO] check_graph: Checking variable coverage...
[INFO] check_graph: Checking edge attributes...
[INFO] check_graph: Checking self-loops...
[INFO] check_graph: Checking graph symmetry...
[INFO] PairedSCGLUEModel: Setting `graph_batch_size` = 4737
[INFO] PairedSCGLUEModel: Setting `max_epochs` = 790
[INFO] PairedSCGLUEModel: Setting `patience` = 66
[INFO] PairedSCGLUEModel: Setting `reduce_lr_patience` = 33
[INFO] PairedSCGLUETrainer: Using training directory: "glue/pretrain"
[INFO] PairedSCGLUETrainer: [Epoch 10] train={'g_nll': 0.624, 'g_kl': 0.024, 'g_elbo': 0.648, 'x_rna_nll': 0.395, 'x_rna_kl': 0.011, 'x_rna_elbo': 0.406, 'x_atac_nll': 0.626, 'x_atac_kl': 0.009, 'x_atac_elbo': 0.635, 'dsc_loss': 0.692, 'vae_loss': 1.113, 'gen_loss': 1.079, 'joint_cross_loss': 1.017, 'real_cross_loss': 1.027, 'cos_loss': 0.304}, val={'g_nll': 0.626, 'g_kl': 0.025, 'g_elbo': 0.651, 'x_rna_nll': 0.397, 'x_rna_kl': 0.01, 'x_rna_elbo': 0.407, 'x_atac_nll': 0.637, 'x_atac_kl': 0.008, 'x_atac_elbo': 0.64

In [ ]:
df_results = pd.DataFrame(
    results,
    columns = ["donor", "foscttm", "label_transfer_accuracy", "alignment_score", "rmse", "nmi", "asw_label", "ari", "asw_batch", "graph_connectivity"]
)
df_results.to_csv('output/scglue_2.csv', index = False)

In [37]:
df_results

,donor,foscttm,label_transfer_accuracy,alignment_score,rmse,nmi,asw_label,ari,asw_batch,graph_connectivity
0,s1d2,0.0422,0.842285,0.956476,0.169282,0.700974,0.585814,0.648648,0.910727,0.951778
1,s1d3,0.0326,0.843421,0.968863,0.149477,0.718436,0.570090,0.513514,0.937366,0.972907
2,s2d1,0.0815,0.860190,0.900669,0.168181,0.724143,0.582817,0.735471,0.888634,0.905477
3,s2d4,0.0582,0.872689,0.931155,0.125049,0.751659,0.614832,0.707645,0.900886,0.927510
4,s2d5,0.1006,0.902758,0.874909,0.138881,0.773474,0.620914,0.814020,0.868497,0.923431
5,s3d10,0.0426,0.869046,0.931326,0.134223,0.795790,0.625322,0.720174,0.935331,0.967390
6,s3d3,0.0356,0.851561,0.971671,0.172021,0.779735,0.590462,0.642080,0.958223,0.972580
7,s3d6,0.0859,0.814175,0.970419,0.260126,0.640087,0.538489,0.572510,0.920650,0.944417
8,s4d8,0.0535,0.878595,0.944370,0.130804,0.717416,0.588160,0.572616,0.932714,0.982795
9,s4d9,0.0424,0.825896,0.963974,0.141999,0.765939,0.585352,0.707864,0.942531,0.964065
